In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from numba import njit, prange
%matplotlib inline

In [ ]:
def power_spectrum(x, step, dx, field, nblock):
    
    n = len(x)    
    dk = 2.0*np.pi/(len(x)*dx)
    p = np.zeros(int(n/2))
     
    for i in range(1,int(n/2)): 
        k = dk*i
        
        suma = 0.
        dtheta = np.pi/90. # two degrees
        
        for j in range (1,180):
            theta = j * dtheta
            
            nx = int( k*np.cos(theta)/dk - n/2)
            ny = int( k*np.sin(theta)/dk - n/2)
            
            suma += x[nx,ny] 
            
        p[i] = suma/180.
        
    filename = "power_" + field + "_" + str(step) + "_nb_" + str(nblock)  + ".txt"         
    np.savetxt(filename, p, delimiter="\n")

In [ ]:
end_step = 100
end_block = 2
dx=1.0

t0 = time.time()
for nb in range(1, end_block):
    print("nb=",nb)
    for step in range(0,end_step):
        print("step=",step)
        c_files = "matrix_c_" + str(step) + "_nb_" + str(nb) + ".txt" 
        c_time  = np.loadtxt(c_files, float)
        
        c_dft = np.fft.fft2(c_time)
        c_dft[0, 0] = 0.0      # Remove the first peak of Fourier coefficients
        c_dft = np.fft.fftshift(c_dft) # Shift to center of the spectrum
        Ac_dft = np.abs(c_dft) # Absolut value of Fourier coefficients   
    
        power_spectrum(Ac_dft, step, dx, "c", nb)
        
        conc_dft_file = "matrix" + "_c_dft_" + str(step) + "_nb_" + str(nb) + ".txt"
        np.savetxt(conc_dft_file, Ac_dft, delimiter="\t")
        
        plt.imshow(Ac_dft, cmap=plt.cm.seismic)
        plt.show()
tf = time.time()
print(tf-t0)

In [ ]:
end_step = 100
end_block = 2
dx = 1.0

t0 = time.time()

for nb in range(1, end_block):
    print("nb=",nb)
    for step in range(0,end_step):
        print("step=",step)
        s_files = "matrix_s_" + str(step) + "_nb_" + str(nb) + ".txt" 
        s_time  = np.loadtxt(s_files, float)
        
        s_dft = np.fft.fft2(s_time)
        s_dft[0, 0] = 0.0      # Remove the first peak of Fourier coefficients
        s_dft = np.fft.fftshift(s_dft) # Shift to center of the spectrum
        As_dft = np.abs(s_dft) # Absolut value of Fourier coefficients   
    
        power_spectrum(As_dft, step, dx, "s", nb)
        
        order_dft_file = "matrix" + "_s_dft_" + str(step) + "_nb_" + str(nb) + ".txt"
        np.savetxt(order_dft_file, As_dft, delimiter="\t")
        
        plt.imshow(As_dft, cmap=plt.cm.seismic)
        plt.show()
tf = time.time()
print(tf-t0)